In [1]:
import arxiv
import logging
import pandas as pd
import datetime

# do logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [2]:
'b'.join(['a'])

'a'

In [3]:
def fetch_recent_papers(categories=["cs.AI"], days=1, max_results=100):
    """
    Fetches papers from specific categories within a time window.
    """
    # 1. Build Query (e.g., "cat:cs.LG OR cat:cs.AI")
    query = " OR ".join([f"cat:{c}" for c in categories])
    
    client = arxiv.Client(
        page_size=100,
        delay_seconds=3.0, # Be nice to ArXiv servers
        num_retries=3
    )
    
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )
    
    # 2. Time Window
    threshold = datetime.datetime.now(datetime.timezone.utc) - datetime.timedelta(days=days)
    
    results = []
    for result in client.results(search):
        # ArXiv results are sorted by date, so we can break early
        if result.published < threshold:
            break
            
        results.append({
            "id": result.entry_id.split('/')[-1],
            "title": result.title,
            "authors": [a.name for a in result.authors],
            "abstract": result.summary.replace("\n", " "),
            "published": result.published,
            "primary_category": result.primary_category,
            "url": result.pdf_url,
        })
        
    print(f"Fetched {len(results)} new papers since {threshold.strftime('%Y-%m-%d %H:%M')}")
    return pd.DataFrame(results)

In [4]:
df = fetch_recent_papers(max_results=1000)
df['combined_text'] = "Title: " + df['title'] + " Abstract: " + df['abstract']

INFO:arxiv:Requesting page (first: True, try: 0): https://export.arxiv.org/api/query?search_query=cat%3Acs.AI&id_list=&sortBy=submittedDate&sortOrder=descending&start=0&max_results=100
INFO:arxiv:Got first page: 100 of 160178 total results
INFO:arxiv:Sleeping: 2.973840 seconds
INFO:arxiv:Requesting page (first: False, try: 0): https://export.arxiv.org/api/query?search_query=cat%3Acs.AI&id_list=&sortBy=submittedDate&sortOrder=descending&start=100&max_results=100


Fetched 193 new papers since 2026-01-29 05:00


In [5]:
df

,id,title,authors,abstract,published,primary_category,url,combined_text
0,2601.22159v1,RedSage: A Cybersecurity Generalist LLM,"[Naufal Suryanto, Muzammal Naseer, Pengfei Li,...",Cybersecurity operations demand assistant LLMs...,2026-01-29 18:59:57+00:00,cs.CR,https://arxiv.org/pdf/2601.22159v1,Title: RedSage: A Cybersecurity Generalist LLM...
1,2601.22156v1,Hybrid Linear Attention Done Right: Efficient ...,"[Yingfa Chen, Zhen Leng Thai, Zihan Zhou, Zhu ...","Hybrid Transformer architectures, which combin...",2026-01-29 18:59:53+00:00,cs.CL,https://arxiv.org/pdf/2601.22156v1,Title: Hybrid Linear Attention Done Right: Eff...
2,2601.22154v1,Exploring Reasoning Reward Model for Agents,"[Kaixuan Fan, Kaituo Feng, Manyuan Zhang, Tian...",Agentic Reinforcement Learning (Agentic RL) ha...,2026-01-29 18:59:52+00:00,cs.AI,https://arxiv.org/pdf/2601.22154v1,Title: Exploring Reasoning Reward Model for Ag...
3,2601.22149v1,DynaWeb: Model-Based Reinforcement Learning of...,"[Hang Ding, Peidong Liu, Junqiao Wang, Ziwei J...","The development of autonomous web agents, powe...",2026-01-29 18:59:07+00:00,cs.CL,https://arxiv.org/pdf/2601.22149v1,Title: DynaWeb: Model-Based Reinforcement Lear...
4,2601.22141v1,Routing the Lottery: Adaptive Subnetworks for ...,"[Grzegorz Stefanski, Alberto Presta, Michal Byra]","In pruning, the Lottery Ticket Hypothesis posi...",2026-01-29 18:56:41+00:00,cs.AI,https://arxiv.org/pdf/2601.22141v1,Title: Routing the Lottery: Adaptive Subnetwor...
...,...,...,...,...,...,...,...,...
188,2601.21283v1,DUET: Distilled LLM Unlearning from an Efficie...,"[Yisheng Zhong, Zhengbang Yang, Zhuangdi Zhu]",LLM unlearning is a technique to remove the im...,2026-01-29 05:32:35+00:00,cs.LG,https://arxiv.org/pdf/2601.21283v1,Title: DUET: Distilled LLM Unlearning from an ...
189,2601.21279v1,NEXUS: Bit-Exact ANN-to-SNN Equivalence via Ne...,[Zhengzheng Tang],Spiking Neural Networks (SNNs) promise energy-...,2026-01-29 05:23:56+00:00,cs.NE,https://arxiv.org/pdf/2601.21279v1,Title: NEXUS: Bit-Exact ANN-to-SNN Equivalence...
190,2601.21278v1,GeoRC: A Benchmark for Geolocation Reasoning C...,"[Mohit Talreja, Joshua Diao, Jim Thannikary Ja...",Vision Language Models (VLMs) are good at reco...,2026-01-29 05:18:40+00:00,cs.CV,https://arxiv.org/pdf/2601.21278v1,Title: GeoRC: A Benchmark for Geolocation Reas...
191,2601.21276v1,"More Code, Less Reuse: Investigating Code Qual...","[Haoming Huang, Pongchai Jaisri, Shota Shimizu...",Large Language Model (LLM) Agents are advancin...,2026-01-29 05:13:21+00:00,cs.SE,https://arxiv.org/pdf/2601.21276v1,"Title: More Code, Less Reuse: Investigating Co..."


In [6]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/Users/yang/Desktop/open_source/arxiv_recommender/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:httpx:HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json "HTTP/1.1 307 Temporary Redirect"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/modules.json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json "HTTP/1.1 307 Temporary Redirec

In [7]:
my_interest_embedding = model.encode("I am interested in Retrieval-augmented generation (RAG).")
content_embedding = model.encode(df['combined_text'].tolist())

print(np.shape(content_embedding))

cosine_sim = np.dot(content_embedding, my_interest_embedding) / (np.linalg.norm(content_embedding, axis=1) * np.linalg.norm(my_interest_embedding))
top_k_indices = np.argsort(-cosine_sim)[:10]

Batches: 100%|██████████| 7/7 [00:01<00:00,  5.47it/s]

(193, 384)


In [11]:
top_df = df.iloc[top_k_indices]

In [43]:
import ollama
import json

def get_authors(author_list):
    """
    Formats the author list for LLM input, truncating if too long.
    """
    # Assuming 'authors' is your list of strings
    num_authors = len(author_list)

    if num_authors <= 12:
        # If list is small, just give them all
        author_string = ", ".join(author_list)
    else:
        # Grab first 10 and last 2 (senior/corresponding positions)
        first_part = author_list[:10]
        last_part = author_list[-2:]
        author_string = f"{', '.join(first_part)} ... {', '.join(last_part)}"
    
    return author_string

def fine_rank_with_llm(top_papers_df, user_interest="RAG"):
    """
    Uses a local LLM to give a qualitative 'Quality Score' to the top candidates.
    """
    top_papers_df = top_papers_df.copy()
    top_papers_df['cleaned_authors'] = top_papers_df['authors'].apply(get_authors)
    all_papers_json = top_papers_df[['id', 'title', 'cleaned_authors', 'abstract']].to_dict(orient='records')
    
    llm_ranking_prompt = f"""
    ## Task

    You are an expert AI and Physics PhD Researcher. 
    Your job is to help me decide which research papers to read based on my specific interest.
    The methodology is to evaluate the title and abstract of each paper and rank them, from best to worst.
    The ranking should consider relevancy, novelty, and soundness of methodology.
    Use the author list to gauge credibility if needed.
    
    My research interest topics: "{user_interest}"

    ## Input paper
    {all_papers_json}

    ## Output

    Provide a JSON response with:
    1. "id": The ArXiv ID of the paper.
    4. "reasoning": A one-sentence explanation of why this paper is worth reading or not.
    """
    
    response = ollama.generate(
        model='llama3:8b', 
        prompt=llm_ranking_prompt,
        format='json', # Ensures we get a parsable dict back
        options={'temperature': 0} # Keep it deterministic
    )
    
    llm_output = json.loads(response['response'])

    refined_results = []
    for paper_analysis in llm_output[:5]:
        paper_id = paper_analysis['id']
        matching_paper = top_papers_df[top_papers_df['id'] == paper_id].iloc[0]
        
        refined_results.append({
            "id": paper_id,
            "url": matching_paper['url'],
            "title": matching_paper['title'],
            "authors": matching_paper['cleaned_authors'],
            "abstract": matching_paper['abstract'],
            "reasoning": paper_analysis.get('why', '')
        })

    return pd.DataFrame(refined_results)

In [44]:
top_df

,id,title,authors,abstract,published,primary_category,url,combined_text
175,2601.21340v1,EHR-RAG: Bridging Long-Horizon Structured Elec...,"[Lang Cao, Qingyu Chen, Yue Guo]",Electronic Health Records (EHRs) provide rich ...,2026-01-29 07:06:34+00:00,cs.AI,https://arxiv.org/pdf/2601.21340v1,Title: EHR-RAG: Bridging Long-Horizon Structur...
51,2601.21916v1,JADE: Bridging the Strategic-Operational Gap i...,"[Yiqun Chen, Erhan Zhang, Tianyi Hu, Shijie Wa...",The evolution of Retrieval-Augmented Generatio...,2026-01-29 16:06:44+00:00,cs.AI,https://arxiv.org/pdf/2601.21916v1,Title: JADE: Bridging the Strategic-Operationa...
52,2601.21912v1,ProRAG: Process-Supervised Reinforcement Learn...,"[Zhao Wang, Ziliang Zhao, Zhicheng Dou]",Reinforcement learning (RL) has become a promi...,2026-01-29 16:04:59+00:00,cs.AI,https://arxiv.org/pdf/2601.21912v1,Title: ProRAG: Process-Supervised Reinforcemen...
27,2601.22025v1,"When ""Better"" Prompts Hurt: Evaluation-Driven ...",[Daniel Commey],Evaluating Large Language Model (LLM) applicat...,2026-01-29 17:32:34+00:00,cs.CL,https://arxiv.org/pdf/2601.22025v1,"Title: When ""Better"" Prompts Hurt: Evaluation-..."
41,2601.21969v1,Token-Guard: Towards Token-Level Hallucination...,"[Yifan Zhu, Huiqiang Rong, Haoran Luo]",Large Language Models (LLMs) often hallucinate...,2026-01-29 16:48:47+00:00,cs.CL,https://arxiv.org/pdf/2601.21969v1,Title: Token-Guard: Towards Token-Level Halluc...
47,2601.21937v1,Retrieval-Infused Reasoning Sandbox: A Benchma...,"[Shuangshuang Ying, Zheyu Wang, Yunjian Peng, ...",Despite strong performance on existing benchma...,2026-01-29 16:26:19+00:00,cs.AI,https://arxiv.org/pdf/2601.21937v1,Title: Retrieval-Infused Reasoning Sandbox: A ...
101,2601.21654v1,ScholarGym: Benchmarking Deep Research Workflo...,"[Hao Shen, Hang Yang, Zhouhong Gu]",Tool-augmented large language models have adva...,2026-01-29 12:51:44+00:00,cs.AI,https://arxiv.org/pdf/2601.21654v1,Title: ScholarGym: Benchmarking Deep Research ...
45,2601.21947v1,ToolWeaver: Weaving Collaborative Semantics fo...,"[Bowen Fang, Wen Ye, Yunyue Su, Jinghao Zhang,...",Prevalent retrieval-based tool-use pipelines s...,2026-01-29 16:29:53+00:00,cs.AI,https://arxiv.org/pdf/2601.21947v1,Title: ToolWeaver: Weaving Collaborative Seman...
89,2601.21714v1,E-mem: Multi-agent based Episodic Context Reco...,"[Kaixiang Wang, Yidan Lin, Jiong Lou, Zhaojiac...",The evolution of Large Language Model (LLM) ag...,2026-01-29 13:42:42+00:00,cs.AI,https://arxiv.org/pdf/2601.21714v1,Title: E-mem: Multi-agent based Episodic Conte...
144,2601.21468v1,MemOCR: Layout-Aware Visual Memory for Efficie...,"[Yaorui Shi, Shugui Liu, Yu Yang, Wenyu Mao, Y...",Long-horizon agentic reasoning necessitates ef...,2026-01-29 09:47:17+00:00,cs.AI,https://arxiv.org/pdf/2601.21468v1,Title: MemOCR: Layout-Aware Visual Memory for ...


In [ ]:
refined_df = fine_rank_with_llm(top_df)

In [33]:
refined_df.sort_values(by=['relevance_score', 'novelty_score', 'soundness_score'], ascending=False)

,id,title,authors,abstract,published,primary_category,url,combined_text,relevance_score,novelty_score,soundness_score,why
0,2601.21340v1,EHR-RAG: Bridging Long-Horizon Structured Elec...,"[Lang Cao, Qingyu Chen, Yue Guo]",Electronic Health Records (EHRs) provide rich ...,2026-01-29 07:06:34+00:00,cs.AI,https://arxiv.org/pdf/2601.21340v1,Title: EHR-RAG: Bridging Long-Horizon Structur...,8,9,9,This paper proposes a novel framework for retr...
5,2601.21937v1,Retrieval-Infused Reasoning Sandbox: A Benchma...,"[Shuangshuang Ying, Zheyu Wang, Yunjian Peng, ...",Despite strong performance on existing benchma...,2026-01-29 16:26:19+00:00,cs.AI,https://arxiv.org/pdf/2601.21937v1,Title: Retrieval-Infused Reasoning Sandbox: A ...,8,9,9,This paper proposes a novel benchmark for eval...
1,2601.21916v1,JADE: Bridging the Strategic-Operational Gap i...,"[Yiqun Chen, Erhan Zhang, Tianyi Hu, Shijie Wa...",The evolution of Retrieval-Augmented Generatio...,2026-01-29 16:06:44+00:00,cs.AI,https://arxiv.org/pdf/2601.21916v1,Title: JADE: Bridging the Strategic-Operationa...,8,9,7,This paper proposes a novel framework for join...
2,2601.21912v1,ProRAG: Process-Supervised Reinforcement Learn...,"[Zhao Wang, Ziliang Zhao, Zhicheng Dou]",Reinforcement learning (RL) has become a promi...,2026-01-29 16:04:59+00:00,cs.AI,https://arxiv.org/pdf/2601.21912v1,Title: ProRAG: Process-Supervised Reinforcemen...,8,9,7,This paper proposes a novel process-supervised...
6,2601.21654v1,ScholarGym: Benchmarking Deep Research Workflo...,"[Hao Shen, Hang Yang, Zhouhong Gu]",Tool-augmented large language models have adva...,2026-01-29 12:51:44+00:00,cs.AI,https://arxiv.org/pdf/2601.21654v1,Title: ScholarGym: Benchmarking Deep Research ...,8,9,7,This paper proposes a novel simulation environ...
7,2601.21947v1,ToolWeaver: Weaving Collaborative Semantics fo...,"[Bowen Fang, Wen Ye, Yunyue Su, Jinghao Zhang,...",Prevalent retrieval-based tool-use pipelines s...,2026-01-29 16:29:53+00:00,cs.AI,https://arxiv.org/pdf/2601.21947v1,Title: ToolWeaver: Weaving Collaborative Seman...,8,9,7,This paper proposes a novel generative tool le...
8,2601.21714v1,E-mem: Multi-agent based Episodic Context Reco...,"[Kaixiang Wang, Yidan Lin, Jiong Lou, Zhaojiac...",The evolution of Large Language Model (LLM) ag...,2026-01-29 13:42:42+00:00,cs.AI,https://arxiv.org/pdf/2601.21714v1,Title: E-mem: Multi-agent based Episodic Conte...,8,9,7,This paper proposes a novel framework for epis...
9,2601.21468v1,MemOCR: Layout-Aware Visual Memory for Efficie...,"[Yaorui Shi, Shugui Liu, Yu Yang, Wenyu Mao, Y...",Long-horizon agentic reasoning necessitates ef...,2026-01-29 09:47:17+00:00,cs.AI,https://arxiv.org/pdf/2601.21468v1,Title: MemOCR: Layout-Aware Visual Memory for ...,8,9,7,This paper proposes a novel multimodal memory ...
4,2601.21969v1,Token-Guard: Towards Token-Level Hallucination...,"[Yifan Zhu, Huiqiang Rong, Haoran Luo]",Large Language Models (LLMs) often hallucinate...,2026-01-29 16:48:47+00:00,cs.CL,https://arxiv.org/pdf/2601.21969v1,Title: Token-Guard: Towards Token-Level Halluc...,8,7,9,This paper proposes a novel approach to contro...
3,2601.22025v1,"When ""Better"" Prompts Hurt: Evaluation-Driven ...",[Daniel Commey],Evaluating Large Language Model (LLM) applicat...,2026-01-29 17:32:34+00:00,cs.CL,https://arxiv.org/pdf/2601.22025v1,"Title: When ""Better"" Prompts Hurt: Evaluation-...",8,6,9,This paper proposes a novel evaluation workflo...


In [21]:
ollama.generate?

Signature:
ollama.generate(
    model: str = '',
    prompt: Optional[str] = None,
    suffix: Optional[str] = None,
    *,
    system: Optional[str] = None,
    template: Optional[str] = None,
    context: Optional[Sequence[int]] = None,
    stream: bool = False,
    think: Optional[bool] = None,
    logprobs: Optional[bool] = None,
    top_logprobs: Optional[int] = None,
    raw: Optional[bool] = None,
    format: Union[Literal['', 'json'], dict[str, Any], NoneType] = None,
    images: Optional[Sequence[Union[str, bytes, ollama._types.Image]]] = None,
    options: Union[Mapping[str, Any], ollama._types.Options, NoneType] = None,
    keep_alive: Union[float, str, NoneType] = None,
) -> Union[ollama._types.GenerateResponse, collections.abc.Iterator[ollama._types.GenerateResponse]]
Docstring:
Create a response using the requested model.

Raises `RequestError` if a model is not provided.

Raises `ResponseError` if the request could not be fulfilled.

Returns `GenerateResponse` if `stream

In [ ]:
)